In [46]:
import numpy as np
#from numpy import genfromtxt
import pandas as pd
df = pd.read_csv("matrix.txt", header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df_transposed = df.T

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_transposed, test_size=0.2, random_state=33)

In [55]:
X_train = X_train.to_numpy()

In [49]:
#cosine distance function
def distCosine(vecA,vecB):
    normA = np.linalg.norm(vecA)
    normB = np.linalg.norm(vecB)
    sims = np.dot(vecA,vecB)/(normA * normB)
    dists = 1 - sims
    return dists

In [56]:
distCosine(X_train[3], X_train[4])

0.9525599047148916

In [57]:
import random
def randCent(dataSet, k):
    n = np.shape(dataSet)[1]
    centroids = np.zeros((k,n), dtype=float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * np.random.rand(k)
    return centroids

In [58]:
randCent(X_train, 3)

array([[ 0.27270361,  0.14082132,  0.84623144, ...,  0.08307759,
         0.36796792,  1.33330313],
       [ 0.11133273,  0.94614743, 33.58865571, ...,  0.56727159,
         0.23077985,  0.57054417],
       [ 0.7572359 ,  0.29847629, 25.34331693, ...,  1.26046215,
         0.64694345,  2.70539815]])

In [74]:
from numpy import *

# kMeans function
def kMeans(dataSet, k, distMeas=distCosine, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = zeros((m,2)) #create mat to assign data points 
                                  #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m): #for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        # print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment

In [75]:
kMeans(X_train, 3)

(array([[0.        , 0.        , 0.        , ..., 0.16666667, 0.        ,
         0.        ],
        [0.00251636, 0.00201309, 0.0367388 , ..., 0.        , 0.        ,
         0.        ],
        [0.28571429, 0.        , 0.        , ..., 0.28571429, 0.14285714,
         0.85714286]]), array([[1.        , 0.80163461],
        [1.        , 0.74152374],
        [1.        , 0.7267203 ],
        ...,
        [1.        , 0.36955623],
        [1.        , 0.52470299],
        [1.        , 0.57512341]]))

In [79]:
terms = pd.read_csv("terms.txt", header=None)
terms

,0
0,aa
1,aargh
2,aaron
3,aaronc
4,ab
...,...
9323,zubov
9324,zv
9325,zw
9326,zx


In [80]:
centroids, clusters = kMeans(X_train, 3)
centroids = pd.DataFrame(centroids, columns=terms)
centroids

,"(aa,)","(aargh,)","(aaron,)","(aaronc,)","(ab,)","(abandon,)","(abc,)","(abid,)","(abil,)","(abl,)",...,"(zq,)","(zr,)","(zrb,)","(zs,)","(zu,)","(zubov,)","(zv,)","(zw,)","(zx,)","(zz,)"
0,0.002558,0.00341,0.056266,0.00682,0.008525,0.006820,0.036658,0.001705,0.038363,0.102302,...,0.004263,0.005115,0.002558,0.000853,0.002558,0.00682,0.026428,0.000000,0.000000,0.003410
1,0.004872,0.00000,0.008526,0.00000,0.008526,0.002436,0.001218,0.000000,0.017052,0.071864,...,0.004872,0.001218,0.000000,0.002436,0.001218,0.00000,0.001218,0.002436,0.001218,0.002436
2,0.000000,0.00000,0.000000,0.00000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.000000,2.000000,0.000000,0.00000,0.000000,0.166667,0.000000,0.000000


In [82]:
doc_clusters = pd.DataFrame(clusters, columns=["Cluster", "MinDistance**2"])
doc_clusters

,Cluster,MinDistance**2
0,1.0,0.730531
1,1.0,0.748543
2,1.0,0.688043
3,1.0,0.670603
4,0.0,0.655568
...,...,...
1995,1.0,0.596127
1996,0.0,0.661816
1997,0.0,0.316578
1998,1.0,0.424407


In [111]:
target = pd.read_csv("classes.txt", header=None)
target

,0
0,0 0
1,1 1
2,2 1
3,3 1
4,4 2
...,...
2495,2495 4
2496,2496 4
2497,2497 3
2498,2498 4


In [109]:
target1 = target.to_numpy()
target1 = target1.ravel()
target1 = np.random.choice(target1,2000)

In [110]:
newC = doc_clusters["Cluster"].astype(int)
print(completeness_score(target1,newC))
print(homogeneity_score(target1,newC))

0.6956138244090708
0.06802078373544375


In [119]:
#Predicted class labels for test data and cosine similarity distance correspoding to that cluster
X_test = X_test.to_numpy()
centroids, clusters = kMeans(X_test, 3)
doc_clusters = pd.DataFrame(clusters, columns=["Class Label", "Cosine Similarity Distance"])
doc_clusters

,Class Label,Cosine Similarity Distance
0,2.0,0.648950
1,2.0,0.603727
2,2.0,0.688933
3,2.0,0.439409
4,0.0,0.360876
...,...,...
495,2.0,0.491609
496,2.0,0.677954
497,2.0,0.727764
498,2.0,0.474880
